In [2]:
import pandas as pd

In [3]:
# import a pickle file
pd_pickle = pd.read_pickle("../pd/2025-03-26_15-13-39_left.pkl")

In [4]:
# add a new column named acc_sum_squared with the sum of the squared values of AccX, AccY, and AccZ
pd_pickle['acc_sum_squared'] = pd_pickle['AccX'] ** 2 + pd_pickle['AccY'] ** 2 + pd_pickle['AccZ'] ** 2

In [5]:
# add 1.5 second to every timestamp in the data
pd_pickle['timestamp'] = pd_pickle['timestamp'] + pd.Timedelta(seconds=1.5)

In [6]:
# delete the rows with timestamp before '2025-03-26 15:05:25.000'
pd_pickle = pd_pickle.iloc[pd_pickle[pd_pickle['timestamp'] == '2025-03-26 15:05:25.000'].index[0]:]

In [7]:
def cut(data, start, end):
    return data[(data['timestamp'] >= start) & (data['timestamp'] <= end)]

In [8]:
# # delete all row after this row with timestamp
# pd_pickle = pd_pickle.iloc[:pd_pickle[pd_pickle['timestamp'] == ''].index[0]]
# # delete all row before this row with timestamp
# pd_pickle = pd_pickle.iloc[pd_pickle[pd_pickle['timestamp'] == ''].index[0]:]

In [9]:
def sure_frequency(data: pd.DataFrame):
    i = 0
    # create an empty list
    frequencies = []
    while True:
        # Calculate the time span in seconds
        time_span = (data['timestamp'].iloc[-(1 + i * 200)] - data['timestamp'].iloc[-(201 + i * 200)]).total_seconds()

        # get the number of records
        number_of_records = 200

        # Calculate the frequency
        frequency = number_of_records / time_span

        # print time span and frequency
        print(f"Time Span: {time_span} seconds")
        print(f"Frequency: {frequency} Hz")
        frequencies.append(frequency)
        i += 1
        if 201 + i * 200 >= len(data):
            break
    # list all different value and corresponding number in frequencies
    from collections import Counter
    print(Counter(frequencies))

In [10]:
print(pd_pickle.shape)
print(pd_pickle.dtypes)
print(pd_pickle.describe())
print(pd_pickle.info)
print(pd_pickle.head())

(99241, 11)
AccX                      float64
AccY                      float64
AccZ                      float64
AsX                       float64
AsY                       float64
AsZ                       float64
AngX                      float64
AngY                      float64
AngZ                      float64
timestamp          datetime64[ns]
acc_sum_squared           float64
dtype: object
               AccX          AccY          AccZ           AsX           AsY  \
count  99241.000000  99241.000000  99241.000000  99241.000000  99241.000000   
mean      -0.022900     -0.013821      0.784907      1.939548      1.648633   
min      -15.946777    -15.000977     -9.305176   -327.026367   -496.520996   
25%       -0.089355      0.001465      0.755371      0.427246     -1.037598   
50%        0.005859      0.008301      0.803711      1.892090      0.732422   
75%        0.047852      0.026855      0.844238      3.967285      2.380371   
max       15.038574      2.434082      7.501953

In [11]:
pd_pickle['timestamp'].diff().describe()
sure_frequency(pd_pickle)

Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 

In [37]:
correct = ('2025-03-26 15:05:28.000', '2025-03-26 15:08:51.000')

In [40]:
wrong_right = ('2025-03-26 15:09:25.000', '2025-03-26 15:11:19.000')

In [17]:
wrong_left = ('2025-03-26 15:11:43.000', '2025-03-26 15:13:32.000')

In [46]:
import matplotlib.pyplot as plt

def show_column(data, column_name, time_range):
    """
    Plots the specified column data interactively within a given time range.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the data.
        column_name (str): The column to be plotted.
        time_range (tuple): A tuple containing the start and end timestamps as strings.
    """
    start_time, end_time = time_range
    filtered_data = data[(data['timestamp'] >= start_time) & (data['timestamp'] <= end_time)]

    %matplotlib notebook
    plt.figure(figsize=(10, 6))
    plt.plot(filtered_data['timestamp'], filtered_data[column_name], label=column_name, color='blue')
    plt.title(f'{column_name} over Time')
    plt.xlabel('Timestamp')
    plt.ylabel(column_name)
    plt.legend()
    plt.grid(True)
    plt.show()


In [45]:
def find_max_value_index(df, input_timestamp, column_name, time_window=2):
    """
    Finds the index of the row with the maximum value in the specified column within a given time window before the given timestamp.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the sensor data.
        input_timestamp (str or pd.Timestamp): The reference timestamp.
        column_name (str): The column to search for the maximum value.
        time_window (int): The time window in seconds before the input timestamp to search within. Default is 2 seconds.

    Returns:
        int or None: The index of the row with the maximum value in the specified column within the time range, or None if no such row exists.
    """
    # Ensure the timestamp is in datetime format
    input_timestamp = pd.to_datetime(input_timestamp)

    # Filter data within the specified time window before the input timestamp
    mask = (df['timestamp'] < input_timestamp) & (df['timestamp'] >= input_timestamp - pd.Timedelta(seconds=time_window))
    filtered_df = df[mask]

    # Find the index of the row with the maximum value in the specified column
    if not filtered_df.empty:
        return filtered_df[column_name].idxmax()
    else:
        return None

In [14]:
# create a list of timestamps
correct_touch = ['2025-03-26 15:06:17.000', '2025-03-26 15:06:38.000', '2025-03-26 15:06:5.000',
                 '2025-03-26 15:07:13.000', '2025-03-26 15:07:29.000', '2025-03-26 15:07:43.000', '2025-03-26 15:07:57.000',
                 '2025-03-26 15:08:10.000', '2025-03-26 15:08:18.000', '2025-03-26 15:08:28.000', '2025-03-26 15:08:40.000']

In [29]:
# use the function for correct_touch
# save the result in a list named correct_touch_max_accx_indexs
correct_touch_max_accx_indexs : list = []
for timestamp in correct_touch:
    correct_touch_max_accx_indexs.append(find_max_value_index(pd_pickle, timestamp, 'accX', 2))

In [31]:
# get the timestamps for correct_touch_max_accx_indexs
correct.loc[correct_touch_max_accx_indexs]

,AccX,AccY,AccZ,AsX,AsY,AsZ,AngX,AngY,AngZ,timestamp
43699,11.690918,-4.145508,-7.726074,-71.350098,162.902832,55.236816,17.528687,-4.575806,87.127075,2025-03-26 15:06:16.765
47906,14.385254,-3.173828,-8.121582,-67.749023,59.631348,31.433105,15.260010,-3.933105,83.375244,2025-03-26 15:06:37.800
41158,0.179199,-0.003418,0.837891,-0.305176,6.347656,-2.258301,1.977539,-2.362061,44.406738,2025-03-26 15:06:04.060
54843,5.581055,-3.029785,0.172852,50.415039,-35.217285,66.467285,-12.161865,8.613281,24.697266,2025-03-26 15:07:12.485
58006,2.290039,-2.354492,1.073730,82.824707,-99.182129,158.935547,-13.529663,3.999023,23.488770,2025-03-26 15:07:28.300
60582,12.039551,-3.866211,-4.534668,-48.461914,125.488281,41.992188,14.765625,-10.447998,81.914062,2025-03-26 15:07:41.180
63614,2.826172,-4.340332,1.352539,83.068848,-62.622070,123.291016,-16.748657,13.128662,24.768677,2025-03-26 15:07:56.340
65964,3.903809,-3.334473,0.670898,64.392090,-123.657227,123.291016,-15.858765,11.409302,27.438354,2025-03-26 15:08:08.090
67640,5.893066,-11.144531,2.381836,-183.166504,6.530762,-275.146484,-4.998779,-7.536621,60.397339,2025-03-26 15:08:16.470
69770,5.319824,-1.763184,-0.550293,31.250000,-31.005859,45.532227,-12.963867,11.898193,23.444824,2025-03-26 15:08:27.120


In [43]:
correct_touch_max_acc_sum_squared_indexs = []
for timestamp in correct_touch:
    correct_touch_max_acc_sum_squared_indexs.append(find_max_value_index(pd_pickle, timestamp, 'acc_sum_squared', 2))

In [44]:
correct.loc[correct_touch_max_acc_sum_squared_indexs]

,AccX,AccY,AccZ,AsX,AsY,AsZ,AngX,AngY,AngZ,timestamp,acc_sum_squared
43699,11.690918,-4.145508,-7.726074,-71.350098,162.902832,55.236816,17.528687,-4.575806,87.127075,2025-03-26 15:06:16.765,213.555021
47906,14.385254,-3.173828,-8.121582,-67.749023,59.631348,31.433105,15.260010,-3.933105,83.375244,2025-03-26 15:06:37.800,282.968810
41195,0.081543,0.499512,0.889648,0.427246,1.037598,1.586914,1.818237,-2.318115,44.126587,2025-03-26 15:06:04.245,1.047636
54822,-2.465820,-12.966797,3.938477,-145.751953,126.403809,-352.416992,-5.383301,0.851440,37.161255,2025-03-26 15:07:12.380,189.729689
57990,-8.048828,-13.994141,2.626953,-78.125000,166.748047,-292.419434,-11.332397,-5.328369,26.499023,2025-03-26 15:07:28.220,267.520489
60582,12.039551,-3.866211,-4.534668,-48.461914,125.488281,41.992188,14.765625,-10.447998,81.914062,2025-03-26 15:07:41.180,180.461584
63600,-13.714844,-5.155762,2.423340,-36.132812,135.314941,-162.109375,-16.831055,6.871948,22.445068,2025-03-26 15:07:56.270,220.551394
65964,3.903809,-3.334473,0.670898,64.392090,-123.657227,123.291016,-15.858765,11.409302,27.438354,2025-03-26 15:08:08.090,26.808534
67640,5.893066,-11.144531,2.381836,-183.166504,6.530762,-275.146484,-4.998779,-7.536621,60.397339,2025-03-26 15:08:16.470,164.601951
69761,-11.471191,-3.395020,1.535156,-15.930176,24.353027,-73.181152,-14.710693,9.772339,19.324951,2025-03-26 15:08:27.075,145.471095


In [36]:
wrong_right_touch = ['2025-03-26 15:09:28.000', '2025-03-26 15:09:40.000', '2025-03-26 15:09:51.000',
                     '2025-03-26 15:10:02.200', '2025-03-26 15:10:14.800', '2025-03-26 15:10:28.000', '2025-03-26 15:10:38.000', '2025-03-26 15:10:48.000', '2025-03-26 15:10:58.000',
                     '2025-03-26 15:11:12.000']

In [46]:
wrong_left_touch = ['2025-03-26 15:11:54.000',
                    '2025-03-26 15:12:03.000', '2025-03-26 15:12:11.200', '2025-03-26 15:12:18.500', '2025-03-26 15:12:27.500', '2025-03-26 15:12:39.300', '2025-03-26 15:12:49.300',
                    '2025-03-26 15:13:02.500', '2025-03-26 15:13:15.500', '2025-03-26 15:13:26.500']

In [19]:
# create a function that take in a dataframe and a timestamp and return a dataframe with the data from duration seconds before the timestamp to the timestamp
def get_data_before_timestamp(data: pd.DataFrame, time_touch: str, duration: int) -> pd.DataFrame:
    """
    Get the data from a specified duration before a given timestamp.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the sensor data.
        time_touch (str): The reference timestamp.
        duration (int): The duration in seconds before the timestamp.

    Returns:
        pd.DataFrame: The data from the specified duration before the timestamp.
    """
    # Ensure the timestamp is in datetime format
    time_touch = pd.to_datetime(time_touch)

    # Calculate the start time based on the duration
    start_time = time_touch - pd.Timedelta(seconds=duration)

    # Filter the data within the specified time range
    # only return data of the columns 'AccX', 'AccY', 'AccZ', 'AsX', 'AsY', 'AsZ'
    filtered_data = data[(data['timestamp'] > start_time) & (data['timestamp'] <= time_touch)][['AccX', 'AccY', 'AccZ', 'AsX', 'AsY', 'AsZ']]

    return filtered_data

In [29]:
# use the get_data_before_timestamp function for the correct_touch timestamps to get a list of dataframes
correct_touch_dataframes : list = []
for timestamp in correct_touch:
    correct_touch_dataframes.append(get_data_before_timestamp(pd_pickle, timestamp, 2))

In [21]:
import numpy as np

In [30]:
# turn the list of dataframes into a numpy array
correct_touch_numpy: np.array = np.array(correct_touch_dataframes)

In [31]:
correct_touch_numpy.shape

(11, 400, 6)

In [32]:
# at the second dimension, add a numpy array([0, 0, 0, 0, 0, 0]) to the correct_touch_numpy at the beginning
correct_touch_numpy = np.insert(correct_touch_numpy, 0, np.array([[0, 0, 0, 0, 0, 0]]), axis=1)

In [33]:
correct_touch_numpy.shape

(11, 401, 6)

In [35]:
correct_touch_numpy[1, 0]

array([0., 0., 0., 0., 0., 0.])

In [38]:
# use the get_data_before_timestamp function for the wrong_right_touch timestamps to get a list of dataframes
wrong_right_touch_dataframes: list = []
for timestamp in wrong_right_touch:
    wrong_right_touch_dataframes.append(get_data_before_timestamp(pd_pickle, timestamp, 2))

In [39]:
# turn the list of dataframes into a numpy array
wrong_right_touch_numpy: np.array = np.array(wrong_right_touch_dataframes)

In [40]:
wrong_right_touch_numpy.shape

(10, 400, 6)

In [41]:
# at the second dimension, add a numpy array([1, 0, 0, 0, 0, 0]) to the wrong_right_touch_numpy at the beginning
wrong_right_touch_numpy = np.insert(wrong_right_touch_numpy, 0, np.array([[1, 0, 0, 0, 0, 0]]), axis=1)

In [42]:
wrong_right_touch_numpy.shape

(10, 401, 6)

In [44]:
wrong_right_touch_numpy[0, 0]

array([1., 0., 0., 0., 0., 0.])

In [47]:
# use the get_data_before_timestamp function for the wrong_left_touch timestamps to get a list of dataframes
wrong_left_touch_dataframes: list = []
for timestamp in wrong_left_touch:
    wrong_left_touch_dataframes.append(get_data_before_timestamp(pd_pickle, timestamp, 2))

In [48]:
# turn the list of dataframes into a numpy array
wrong_left_touch_numpy: np.array = np.array(wrong_left_touch_dataframes)

In [49]:
wrong_left_touch_numpy.shape

(10, 400, 6)

In [50]:
# at the second dimension, add a numpy array([2, 0, 0, 0, 0, 0]) to the wrong_left_touch_numpy at the beginning
wrong_left_touch_numpy = np.insert(wrong_left_touch_numpy, 0, np.array([[2, 0, 0, 0, 0, 0]]), axis=1)

In [52]:
wrong_left_touch_numpy.shape

(10, 401, 6)

In [53]:
wrong_left_touch_numpy[0, 0]

array([2., 0., 0., 0., 0., 0.])

In [54]:
# combine the correct_touch_numpy, wrong_right_touch_numpy, and wrong_left_touch_numpy
combined_numpy = np.concatenate((correct_touch_numpy, wrong_right_touch_numpy, wrong_left_touch_numpy))

In [55]:
combined_numpy.shape

(31, 401, 6)

In [64]:
# save the combined_numpy as a .pkl file using pickle
import pickle
with open('../classification_forward/data/combined_numpy.pkl', 'wb') as f:
    pickle.dump(combined_numpy, f)